In [2]:
using BenchmarkTools

In [155]:
include("src/staticexponents.jl")

IJulia.sp

In [156]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

y

In [157]:
t = 3 * x * y

3xy

In [158]:
p = 3x^2 + y

3x^2 + 1y

In [159]:
sp.subs(p.terms[1], x=>2)

12.0

In [160]:
sp.subs(p, x=>2, y=>1)

13.0

In [162]:
@code_warntype sp.subs(p, x=>2, y=>1)

Variables:
  #self#::IJulia.sp.#subs
  p::IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{x^2,Int64},IJulia.sp.Term{y,Int64}}}
  s::Tuple{Pair{IJulia.sp.Variable{:x},Int64},Pair{IJulia.sp.Variable{:y},Int64}}

Body:
  begin  # line 201:
      $(Expr(:inbounds, false))
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl subs 193
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl subs 174
      SSAValue(3) = $(Expr(:invoke, MethodInstance for ^(::Int64, ::Float64), :(Base.^), :((Core.getfield)((Core.getfield)(s, 1)::Pair{IJulia.sp.Variable{:x},Int64}, :second)::Int64), :((Base.div_float)((Base.sitofp)(Float64, (Core.getfield)(2//1, :num)::Int64)::Float64, (Base.sitofp)(Float64, (Core.getfield)(2//1, :den)::Int64)::Float64)::Float64)))
      # meta: pop location
      # meta: pop location
      $(Expr(:inbounds, :pop))
      $(Expr(:inbounds, false))
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/sr

In [104]:
t(x=>2, y=>5)

30.0

In [105]:
@code_warntype t(x=>2, y=>5)

Variables:
  m::IJulia.sp.Term{xy,Int64}
  args::Tuple{Pair{IJulia.sp.Variable{:x},Int64},Pair{IJulia.sp.Variable{:y},Int64}}

Body:
  begin 
      SSAValue(5) = (Core.getfield)(args::Tuple{Pair{IJulia.sp.Variable{:x},Int64},Pair{IJulia.sp.Variable{:y},Int64}}, 1)::Pair{IJulia.sp.Variable{:x},Int64}
      SSAValue(6) = (Core.getfield)(args::Tuple{Pair{IJulia.sp.Variable{:x},Int64},Pair{IJulia.sp.Variable{:y},Int64}}, 2)::Pair{IJulia.sp.Variable{:y},Int64}
      $(Expr(:inbounds, false))
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl subs 190
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl subs 171
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl # line 184:
      SSAValue(2) = $(Expr(:invoke, MethodInstance for ^(::Int64, ::Float64), :(Base.^), :((Core.getfield)(SSAValue(5), :second)::Int64), :((Base.div_float)((Base.sitofp)(Float64, (Core.getfield)(1//1, :num):

In [106]:
t(x=>2)

6.0y

In [107]:
@code_warntype t(x=>2)

Variables:
  m::IJulia.sp.Term{xy,Int64}
  args::Tuple{Pair{IJulia.sp.Variable{:x},Int64}}

Body:
  begin 
      $(Expr(:inbounds, false))
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl subs 171
      # meta: location /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl # line 184:
      SSAValue(1) = $(Expr(:invoke, MethodInstance for ^(::Int64, ::Float64), :(Base.^), :((Core.getfield)((Core.getfield)(args, 1)::Pair{IJulia.sp.Variable{:x},Int64}, :second)::Int64), :((Base.div_float)((Base.sitofp)(Float64, (Core.getfield)(1//1, :num)::Int64)::Float64, (Base.sitofp)(Float64, (Core.getfield)(1//1, :den)::Int64)::Float64)::Float64)))
      SSAValue(0) = $(Expr(:new, :($(QuoteNode(IJulia.sp.Term{y,Float64}))), :((Base.mul_float)((Base.sitofp)(Float64, (Core.getfield)(m, :coefficient)::Int64)::Float64, SSAValue(1))::Float64)))
      # meta: pop location
      # meta: pop location
      $(Expr(:inbounds, :pop))
      return SSAValue(0)

In [64]:
typeof(x => 2)

Pair{IJulia.sp.Variable{:x},Int64}

In [65]:
z = x => 2

x=>2

In [66]:
dump(z)

Pair{IJulia.sp.Variable{:x},Int64}
  first: IJulia.sp.Variable{:x} x
  second: Int64 2


In [63]:
sp.subs(t, x, 2)

6.0y

In [37]:
@code_warntype x * x + y

Variables:
  #self#::Base.#+
  v1::IJulia.sp.Monomial{(x^2,)}
  v2::IJulia.sp.Variable{:y}

Body:
  begin 
      return $(Expr(:new, :($(QuoteNode(IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{x^2,Int64},IJulia.sp.Term{y,Int64}}}))), :((Core.tuple)($(Expr(:new, :($(QuoteNode(IJulia.sp.Term{x^2,Int64}))), 1)), $(Expr(:new, :($(QuoteNode(IJulia.sp.Term{y,Int64}))), 1)))::Tuple{IJulia.sp.Term{x^2,Int64},IJulia.sp.Term{y,Int64}})))
  end::IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{x^2,Int64},IJulia.sp.Term{y,Int64}}}


In [38]:
@benchmark $x^2 + $y

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.694 ns (0.00% GC)
  median time:      1.738 ns (0.00% GC)
  mean time:        1.818 ns (0.00% GC)
  maximum time:     9.446 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [39]:
@benchmark $x^2 + $y + $x * $x + 3 * $x * $y + $x * $y^2

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.984 ns (0.00% GC)
  median time:      2.044 ns (0.00% GC)
  mean time:        2.166 ns (0.00% GC)
  maximum time:     35.285 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1000
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [40]:
x + y

1x + 1y

In [41]:
x * x

x^2

In [42]:
x + x

2x

In [43]:
x * y

xy

In [44]:
1 * (x * y) + 3 * (x * y)

4xy

In [45]:
x^1 + x^1

2x

In [46]:
x * y

xy